# Гетероскедастичность

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/ryupepa/Econometrica/master/python-notebooks/data-csv/sleep75.csv')

In [ ]:
model = smf.ols(data=df, formula='sleep~totwrk+age+I(age**2)+male+smsa+south').fit()

In [ ]:
# Для того, чтобы проверить наличие гетероскедастичности с помощью теста Бройша-Пагана,
# необходимо построить вспомогательную регрессию,
#зависимой переменной в которой будут квадраты остатков исходной регресси.

$$e_i = y_i - \hat{y}_i$$
$$
e_i^2 = \gamma_0 + \gamma_1*totwrk+age+\gamma_2*I(age**2)+\gamma_3*male+\gamma_3*smsa+\gamma_4*south
$$

In [ ]:
# квадраты остатков добавим в датафрейм, воспользовавшись методом '.resid', возвращающий значения остатков
df['res2']=model.resid**2
df['res']=model.resid

In [ ]:
 sub_model= smf.ols(data=df, formula='res2~totwrk+age+I(age**2)+male+smsa+south').fit()

## Сформулируем проверямемую гипотезу, о характере зависимости дисперсии ошибок от факторов регресси
$$
H_0: \sigma_i^2 \equiv \sigma^2=f(\gamma_0),
$$
$$
H_1:\sigma_i^2 = f(\gamma_0 + z_{i1}\gamma_1 + z_{i2}*\gamma_2 + \dots + \gamma_{ip}\gamma_p).
$$
### Другими словами, значимость вспомогательной регресси в целом, т.е.
$$
H_0: \gamma_1=\gamma_2=\gamma_3=\gamma_4=\gamma_5=\gamma_6=0,
$$
$$
H_1:  \gamma_1^2+\gamma_2^2+\gamma_3^2+\gamma_4^2+\gamma_5^2+\gamma_6^2>0.
$$

### При справедливости нулевой гипотезы статистика $n*R_0^2 $ имеет распределение хи-квадрат:
$$
nR_0^2 \approx_{H_0} \chi^2_p
$$

In [ ]:
St = np.round(len(df)*sub_model.rsquared,2)
St

In [ ]:
Hi2 = stats.chi2.ppf(1-0.05,6)
Hi2

$$
nR_0^2 < \chi_p^2.
$$
### Нет оснований отвергнуть нулевую гипотезу. Тест указывает на гомоскедастичность.

# 2.1. Output equation

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/Labour.csv')
df

### В случае гетерскедостичности ошибок регрессии основной недостаток OLS-оценок коэффициентов регрессии состоит в том, что статистические выводы, основанные на применениеи t- и F-статистик, уже неверны. Уайт предложил использовать устойчивые к гетерскедостичности скорректированные стандартные ошибки коэффициентов, так как t-статисктики, вычисленные обычным образом по скорректированным стандартным ошибкам, имеют нужное распределение Стьюдента. Такие робастные оценки называются $HC-$оценками и строятся они по формуле

$\hat{V} =(X'X)^{-1}X'\Omega (X'X)^{-1} $, где $\Omega_{n \times n} = diag(\omega_1,\dots, \omega_n) $.

|  $HC0$ | $\omega_i = e_i^2 $   | 
|---|---|
|  $HC1$ | $\omega_i = \frac{n}{n-k-1} e_i^2 $   | 
|  $HC2$ | $\omega_i = \frac{e_i}{(1-h_{ii})} $   | 
|  $HC3$ | $\omega_i =\frac{e_i}{(1-h_{ii})^2} $   | 
|  $HC4$ | $\omega_i = \frac{e_i}{(1-h_{ii})^{d_i}} $   | 
|  $HC4m$ | $\omega_i =\frac{e_i}{(1-h_{ii})^{\delta_i}} $   | 
|  $HC5$ | $\omega_i =\frac{e_i}{(1-h_{ii})^{\gamma_i^2}} $   | 


Здесь $h_{ii}-$ диагональный элемент матрицы $H = X(X'X)^{-1}X'$
$$
d_i = min\{4, h_{ii}/\bar{h}\},\\
\delta_i = min\{1, h_{ii}/\bar{h}  \} + min\{ 1, 5, h_{ii}/\bar{h} \},\\
\gamma_i = min{h_{ii}/ \bar{h}, ,max\{ 4, 0, 7 h_{max}/\bar{h} \}}.
$$

### Асимптотически все варианты $HC-$оценок эквивалентны. Различие между ники в точности оценивается при малых выборках.

In [ ]:
# Для того, чтобы с помощью библиотеку statsmodels вычислить оценки, устойчивые к гетерскедостичности, 
# необходимо в метод .fit() указать соответсвующий аргумент
model = smf.ols(data=df, formula='np.log(output)~np.log(capital)+np.log(labour)+I(np.log(capital)**2)+I(np.log(labour)**2)').fit(cov_type='HC3')

In [ ]:
# 
model.summary().tables[1]